# Homework 3
In this homework assignment, you will implement a univariate feature selection method. 

You will be given a toy dataset called 'Car Evaluation Data Set' (see: http://archive.ics.uci.edu/ml/datasets/Car+Evaluation for details).
You are not required to, but advised to test your code with the toy dataset, or any other dataset that contains categorical variables.

The given dataset contains six descriptive features and a target variable. Each of those are ordinal scale, categorical variables. The name of the target feature is 'evaluation'. 

Note here that you are expected to write your own code, so DO NOT COPY AND PASTE CODE OR USE LIBRARY FUNCTIONS. The goal of the homework is not to see if you can call library functions but to have you practice with the impurity measures and feature selection techniques.


In [240]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

### Read the dataset

In [241]:
edf = pd.read_csv('careval.csv')
# edf.head()
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   buying      1728 non-null   object
 1   maint       1728 non-null   object
 2   doors       1728 non-null   object
 3   persons     1728 non-null   object
 4   lug_boot    1728 non-null   object
 5   safety      1728 non-null   object
 6   evaluation  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


You will create a method called IUFS (impurity-based univariate feature selection), which will select the most informative features with a univariate feature selection schema. This feature selection method will take the dataset, name of the target variable, number of features to be selected (k) and the measure of impurity as an input, and will output the names of k best features based on the information gain. You are expected to implement information gain, entropy and Gini index functions. Note here that this will be a univariate selection, which means that you need to test the features individually.

In [242]:
# entropy (H)
def entropy(feature, dataset):
    """Calculates the entropy of a feature in a given dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    dataset: pd.DataFrame
        dataframe for the dataset
    Returns
    -------
    float
        entropy for the feature in the dataset
    """
    df = dataset[feature]
    totalValues = df.size
    uniqueValues = df.unique()
    uniqueValuesCount = []
    
    for i in range(uniqueValues.size):
        uniqueValuesCount.append(0)
    
    
    for row in df.iteritems():
        uniqueValuesCount[uniqueValues.tolist().index(row[1])] += 1
    
    numEntropy = 0.0
    length = len(uniqueValuesCount)
    for i in range(length):
        insideLog = uniqueValuesCount[i]/totalValues
        if insideLog != 0:
            numEntropy += ((uniqueValuesCount[i])/totalValues)*np.log2(((uniqueValuesCount[i])/totalValues))
        else:
            numEntropy += 0
    return -numEntropy
    pass

entropy('buying', edf) 


2.0

In [243]:
# gini index (Gini)

def gini(feature, dataset):
    """Calculates the gini index of a feature in a given dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    dataset: pd.DataFrame
        dataframe for the dataset
    Returns
    -------
    float
        gini index for the feature in the dataset
    """
    df = dataset[feature]
    totalValues = df.size
    uniqueValues = df.unique()
    uniqueValuesCount = []
    
    for i in range(uniqueValues.size):
        uniqueValuesCount.append(0)
    
    
    for row in df.iteritems():
        uniqueValuesCount[uniqueValues.tolist().index(row[1])] += 1
        
    giniNum = 1.0
    length = len(uniqueValuesCount)
    for i in range(length):
       giniNum -= ((uniqueValuesCount[i]/totalValues)*(uniqueValuesCount[i]/totalValues))
    return giniNum
    pass

gini('buying', edf) 

0.75

In [244]:
# information gain (IG)

def IG(feature, target, dataset, measure):
    """Calculates the information gain of a feature for a given target variable and a dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    measure: str ('entropy' or 'gini')
        measure of impurity to be used
    Returns
    -------
    float
        information gain for the feature in the dataset for a given target variable
    """
    df = dataset[[feature,target]]
    totalValues = df[feature].size
    if measure == 'entropy':
        targetEntropy = entropy(target,dataset)
        uniqueValues = df[feature].unique()
        targetUnique = df[target].unique()
        uniqueValuesUnCount = []
        uniqueValuesACount = []
        for i in range(uniqueValues.size):
            uniqueValuesUnCount.append(0)
            uniqueValuesACount.append(0)
        for row in df.iterrows():
            targetVal = row[1][target]
            if targetVal == targetUnique[0]:
                uniqueValuesUnCount[uniqueValues.tolist().index(row[1][feature])] = uniqueValuesUnCount[uniqueValues.tolist().index(row[1][feature])]+1
            else: 
                uniqueValuesACount[uniqueValues.tolist().index(row[1][feature])] = uniqueValuesACount[uniqueValues.tolist().index(row[1][feature])]+1
        infoGain = targetEntropy
        length = len(uniqueValuesACount)
        for i in range(length):
            numValuesI = uniqueValuesUnCount[i]+uniqueValuesACount[i]
            infoGain -= (numValuesI/totalValues)*(entropy(target,df.loc[df[feature] == uniqueValues[i]]))
           
        return infoGain
    elif measure == 'gini':
        targetGini = gini(target,dataset)
        uniqueValues = df[feature].unique()
        targetUnique = df[target].unique()
        
        uniqueValuesUnCount = []
        uniqueValuesACount = []
    
        for i in range(uniqueValues.size):
            uniqueValuesUnCount.append(0)
            uniqueValuesACount.append(0)
    
        

        for row in df.iterrows():
            targetVal = row[1][target]
            if targetVal == targetUnique[0]:
                uniqueValuesUnCount[uniqueValues.tolist().index(row[1][feature])] = uniqueValuesUnCount[uniqueValues.tolist().index(row[1][feature])]+1
            else: 
                uniqueValuesACount[uniqueValues.tolist().index(row[1][feature])] = uniqueValuesACount[uniqueValues.tolist().index(row[1][feature])]+1
       
        infoGain = targetGini
        length = len(uniqueValuesACount)
        for i in range(length):
            numValuesI = uniqueValuesUnCount[i]+uniqueValuesACount[i]
            infoGain -= (numValuesI/totalValues)*(gini(target,df.loc[df[feature] == uniqueValues[i]]))
            
        return infoGain
    
    else : return
  
    pass


IG('buying','evaluation', edf, 'entropy') 

0.09644896916961382

In [245]:
def IUFS(target, dataset, k, measure='entropy'):
    """Finds k most informative features in the given dataset based on the target variable
        using information gain with the selected measure.
        
    Parameters
    ----------
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    k: int
        number of features to return, must be less than or equal to number of descriptive features in dataset.
        in other words, 0 < k < len(dataset.columns).
    measure: str, 'entropy' or 'gini'
        measure of impurity
    Returns
    -------
    list
        returns a list of k feature names, selected based on univariate selection schema
    """
    cols = list(dataset.columns.values)
    infoGains = []
    for col in cols:
        if col != target:
            infoGains.append(IG(col,target,dataset,measure))
    
    kMost = sorted(infoGains,reverse=True)
    
    infoFeatures = []
    for i in range(k):
        highest = kMost[i]
        index = infoGains.index(highest)
        infoFeatures.append(cols[index])
    return infoFeatures
    pass

IUFS('evaluation', edf, 2, measure='entropy')

['safety', 'persons']

### Bonus
Improve the IUFS by including an option for gain ratio. Gain ratio is an alternative to information gain and can be used with either of the Gini index or entropy measures.  

In [246]:
def GR(feature, target, dataset, measure):
    """Calculates the gain ratio of a feature for a given target variable and a dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    measure: str ('entropy' or 'gini')
        measure of impurity to be used
    Returns
    -------
    float
        gain ratio for the feature in the dataset for a given target variable
    """
    df = dataset[[feature,target]]
    totalValues = df[feature].size
    infoGain = IG(feature, target, dataset, measure)
    if measure == 'gini'
        return infoGain/gini(feature,dataset)
    elif measure == 'entropy':
        return infoGain/entropy(feature,dataset)
    else:
        return
    pass

    


GR('buying','evaluation', edf, 'gini') 

SyntaxError: invalid syntax (<ipython-input-246-7bdc979c5c42>, line 22)

In [247]:
def IUFS2(target, dataset, k, measure='entropy', gain='IG'):
    """Finds k most informative features in the given dataset based on the target variable
        using information gain with the selected measure.
        
    Parameters
    ----------
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    k: int
        number of features to return, must be less than or equal to number of descriptive features in dataset.
        in other words, 0 < k < len(dataset.columns).
    measure: str, 'entropy' or 'gini'
        measure of impurity
    gain: str, 'IG' or 'GR'
        feature selection metric ('IG' for information gain, 'GR' for gain ratio)
    Returns
    -------
    list
        returns a list of k feature names, selected based on univariate selection schema
    """
    cols = list(dataset.columns.values)
    infoGains = []
    if gain == 'IG':
        for col in cols:
            if col != target:
                infoGains.append(IG(col,target,dataset,measure))

    elif gain == 'GR':
        for col in cols:
            if col != target:
                infoGains.append(GR(col,target,dataset,measure))
    else:
        return
    kMost = sorted(infoGains,reverse=True)

    infoFeatures = []
    for i in range(k):
        highest = kMost[i]
        index = infoGains.index(highest)
        infoFeatures.append(cols[index])
    return infoFeatures
    
    pass


IUFS2('evaluation', edf, 5, measure='gini', gain='IG')

['safety', 'persons', 'buying', 'maint', 'lug_boot']